# Parallel Task Dispatcher

## Role in the Project
This script serves as the **uniform interface** for the parallel generation of the synthetic dataset. It acts as a wrapper that abstracts the differences between the Hadronic and Quark generation routines, allowing the main orchestration script to distribute computational tasks across multiple CPU cores using a single generic function call.

Its primary purpose is to receive a specific workload description (a "task"), determine which physics engine is required (Hadronic mixing or Quark CFL parameters), and route the execution accordingly.

## Physics and Equations
While this script does not solve physical equations directly, it manages the transmission of the **Hadronic Baselines** required for the homologous scaling physics used in the hadronic sector.

The `baselines` data structure contains the maximum stable mass ($M_{max}$) for every parent Equation of State in the library. This information is critical for the "Smart Inverse Sampling" algorithm, which uses the scaling relation:

$$
\alpha = \left( \frac{M_{base}}{M_{target}} \right)^2
$$

to generate new hadronic stars with specific target masses. This script ensures that this shared data is correctly passed to the hadronic workers while being ignored for quark workers (which rely on the CFL parameter space: $B, \Delta, m_s$).

## Algorithm and Calculations
The logic implemented in this wrapper is a **conditional dispatch** mechanism designed for high-throughput parallel processing:

1.  **Task Unpacking**
    The input `task_tuple` is decomposed into its constituent instruction set:
    *   **Mode:** The type of matter to simulate ('hadronic' or 'quark').
    *   **Batch Size:** The number of EoS curves to generate in this specific job.
    *   **Random Seed:** A unique integer to ensure stochastic independence between parallel processes.
    *   **Batch ID:** A label for tracking and organization.

2.  **Conditional Routing**
    *   If the mode is **Hadronic**, the execution is routed to the Hadronic Generator. The `baselines` dictionary is passed explicitly to enable the scaling calculations.
    *   If the mode is **Quark**, the execution is routed to the Quark Generator. The `baselines` are not required, as quark stars are generated using the inverse scaling of the Bag Constant ($B \propto M^{-2}$).

3.  **Result Aggregation**
    The list of generated star vectors returned by the specific worker is passed back up to the main process for aggregation into the master dataset.

## Inputs and Outputs

### Inputs
*   **Task Tuple**: A package containing `(mode, batch_size, seed, batch_id)`.
*   **Baselines**: A dictionary mapping parent model names to their maximum stable masses (calculated in `calculate_baselines.py`).

### Outputs
*   **Star Data List**: A list of feature vectors, where each vector represents a single stable neutron star configuration (Mass, Radius, Tidal Deformability, Microphysics parameters, etc.).

## Connection to the Overall Workflow
This script is the direct link between the **Process Orchestrator** (`main.py`) and the **Physics Engines** (`worker_hadronic_gen.py` and `worker_quark_gen.py`).

In the main pipeline, a list of tasks is created (e.g., "Generate 50 hadronic curves", "Generate 50 quark curves"). This wrapper allows the `joblib` parallel library to execute this heterogeneous list of tasks without needing to manage separate queues for different physics types.